In [ ]:
#Let's implement the 2003 Bag Of Words Paper

In [ ]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

In [ ]:
#Data:
names = open("names.txt").read().splitlines()
print(f"{len(names)=}")

# Build Vocab
itos = {i+1:s for i,s in enumerate(sorted(set("".join(names))))}
itos[0] = '.'
stoi = {s:i for i,s in itos.items()}

In [ ]:
#Data Set Creation:
x, y = [], []
context_len = 3
xchar, ychar = [], []
buff = [0]*context_len
buff = [itos[b] for b in buff]
for name in names:
    augmented_name = buff + list(name) + ['.']
    for i in range(len(augmented_name)-context_len):
        xi = augmented_name[i:i+context_len ]
        yi = augmented_name[i+context_len]
        #print(f"{xi} -------> {yi}")
        xchar.append(xi)
        ychar.append(yi)
        x.append([stoi[d] for d in xi])
        y.append(stoi[yi])

print(len(x)), print(len(y))
print(len(xchar))

In [ ]:
#Create Tensors:
X = torch.tensor(x)
Y = torch.tensor(y)

X.shape, Y.shape

In [ ]:
X[:5,:]

In [ ]:
#Let's create the embeddings
C = torch.rand((27,2))
emb = C[X]
emb.shape

In [ ]:
#Lets create the NN
Xtr = torch.cat(torch.unbind(emb,1),dim=1)
print(Xtr.shape)


In [ ]:
# Layer 1 has 100 neurons
# Layer 2 is Softmax
# 6 --> 100 --> 27 --> Softmax

n1 = 100
n2 = 200
n3 = 27
layers = [n1,n2,n3]

def paramsetting(prev,layers):
        W = []
        b = []
        for layer in layers:
            W.append(torch.rand((prev,layer),requires_grad=True))
            b.append(torch.rand((layer),requires_grad=True))
            prev = layer
        return W,b


def mainloop(X,Y, layers,iterations = 100, step_size = 0.9):
    C = torch.rand((27,2),requires_grad=True)
    W,b = paramsetting(6,layers)
    losses = []

    def forward(C, W,b):
        emb = C[X]
        H = torch.tanh(emb.view(-1,6))
        for Wi,bi in zip(W,b):
            H = torch.tanh(H@Wi + bi)
        loss = F.cross_entropy(H,Y)
        return loss

    def backward(loss):
        params = [C] + W + b
        for param in params:
            param.grad = None
        loss.backward()
        for param in params:
            param.data += -step_size * param.grad
        
        return C, W,b
    
    # return backward(forward(C,W,b))
    
    for _ in range(iterations):
        loss = forward(C,W,b)
        losses.append(loss.data)
        print(f"{loss=}")
        C,W,b = backward(loss)
    
    return losses

#losses = mainloop(X,Y,[100,27])
         

In [ ]:
plt.plot(losses)